<a href="https://colab.research.google.com/github/SharlotteManganye/Deep-Learning-CNN/blob/main/Cfar100_Wider_net_HLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/WinterWen666/Efficient-High-Order-Volterra-Convolution/tree/main

In [1]:

import torch

import numpy as np
import torch.nn as nn
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms
from torch.autograd import Function
# import import_ipynb
# from Efficient_High_order_conv import PM_creation, PCM_creation, HO_conv
from torchvision.transforms.autoaugment import AutoAugmentPolicy


Efficient_High_order_conv

In [2]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def PM_creation( N, input_size, TRM):
    index = []
    PM = torch.arange(input_size)[:,None]
    TPM = []
    TPM.append(PM)
    PM_full = []
    for i in range(N-1):
        Nc = int(PM.shape[1])
        for j in range(input_size-Nc):
            j = j+1
            p_N1 = (PM[:,Nc-1] + j)[:, None]
            index_chosen = torch.argwhere(p_N1 < input_size)
            index_final = torch.concat((PM[index_chosen[:,0]],p_N1[index_chosen[:,0]]), axis = 1)
            index.append(index_final)
            p_N1 = p_N1 - j
        PM = torch.vstack(index)
        index = []
        TPM.append(PM)
    for k in range(len(TRM)):
        PM = TPM[len(TRM[k])-1]
        PM_full.append(np.repeat(PM, TRM[k], axis =1))
    PM_full = torch.vstack(PM_full)
    return PM_full


def PCM_creation(PM_full_r_1, PM_full_r):
    PCM_r = []
    PCMs_r = []
    for i in range(PM_full_r.shape[1]):
        PM_deleted = torch.cat((PM_full_r[:, :i], PM_full_r[:, i + 1:]), dim=1)
        for row_r in PM_deleted:
            Position = torch.where(torch.all(PM_full_r_1 ==row_r, axis = 1))
            PCM_r.append(torch.tensor(Position))
        PCM_r = torch.vstack(PCM_r)
        PCM_r = torch.concat((PM_full_r[:,i][:,None], PCM_r), axis = 1)
        PCMs_r.append(PCM_r)
        PCM_r = []
    return  PCMs_r


class HT_creation(Function):
        @staticmethod
        def forward(ctx, input, kernel_size, stride, padding,
                  TPCMs_r):
            input = input.to(device)
            shape = input.shape
            grad_move = []
            grad_move.append(0)
            x_r_1_move =[]
            x_r_1_move.append(0)
            HT = []
            unfold = nn.Unfold(kernel_size = kernel_size,  padding = padding, stride = stride)
            Col = unfold(input)
            weight_shape = []

            ori_col = int(kernel_size[0]*kernel_size[1])
            Col = Col.view(shape[0], shape[1], ori_col, Col.shape[-1])

            for i in range(len(TPCMs_r)):
                if i == 0:
                    grad_move.append(grad_move[i]+TPCMs_r[i][0].shape[0])
                    weight_shape.append((ori_col, ori_col))
                    HT.append(Col[:,:,TPCMs_r[i][0][:,0],:]*Col[:,:,TPCMs_r[i][0][:,1],:])
                    x_r_1_move.append(x_r_1_move[i]+Col.shape[2])
                else:
                    grad_move.append(grad_move[i]+TPCMs_r[i][0].shape[0])
                    weight_shape.append((TPCMs_r[i-1][0].shape[0], ori_col))
                    HT.append(Col[:,:,TPCMs_r[i][0][:,0],:]*HT[i-1][:,:,TPCMs_r[i][0][:,1],:])
                    x_r_1_move.append(x_r_1_move[i]+TPCMs_r[i-1][0].shape[0])
            HT = torch.cat(HT, dim = 2)
            if len(TPCMs_r) == 1:
                # ctx.back_terms = Col
                ctx.save_for_backward(Col)
            else:
                # ctx.back_terms = torch.cat((Col,HT[:,:,0:grad_move[-2],:]), dim=2 )
                ctx.save_for_backward(torch.cat((Col,HT[:,:,0:grad_move[-2],:]), dim=2 ))
            ctx.grad_move = grad_move
            ctx.x_r_1_move = x_r_1_move
            ctx.TPCMs_r = TPCMs_r

            ctx.weight_shape = weight_shape
            ctx.kernel_size =kernel_size
            ctx.stride = stride
            ctx.padding = padding
            ctx.shape =shape

            return HT
        @staticmethod
        def backward(ctx, grad):
            (back_terms,) = ctx.saved_tensors

            # back_terms = ctx.back_terms
            weight_shape = ctx.weight_shape
            TPCMs_r = ctx.TPCMs_r
            kernel_size = ctx.kernel_size
            stride = ctx.stride
            padding = ctx.padding
            shape = ctx.shape
            grad_move = ctx.grad_move
            x_r_1_move = ctx.x_r_1_move
            grad_input  = None

            for i in range(len(TPCMs_r)):
                Al_grad = torch.zeros((grad.shape[0], grad.shape[1],
                                          weight_shape[i][0], weight_shape[i][1],grad.shape[-1])).to(device)
                if i == 0:
                    Al_grad[:,:, TPCMs_r[0][0][:,0],TPCMs_r[0][0][:,1],:] = grad[:,:,grad_move[0]:grad_move[1],:]
                    # Al_grad[:,:, PCMs_r[0][:,1],PCMs_r[0][:,0],:] += grad[:,:,grad_move[0]:grad_move[1],:]
                    Al_grad = Al_grad + Al_grad.transpose(2,3)
                    grad_input = torch.einsum('bcikj,bckj->bcij',Al_grad,
                                              back_terms[:,:,x_r_1_move[0]:x_r_1_move[1],:])
                else:
                    Al_grad[:,:, TPCMs_r[i][0][:,1], TPCMs_r[i][0][:,0],:] = grad[:,:,grad_move[i]:grad_move[i+1],:]
                    for j in range(i+1):
                        Al_grad[:,:, TPCMs_r[i][j+1][:,1], TPCMs_r[i][j+1][:,0],:] +=grad[:,:,grad_move[i]:grad_move[i+1],:]
                    grad_input += torch.einsum('bckij,bckj->bcij',Al_grad,
                                                back_terms[:,:,x_r_1_move[i]:x_r_1_move[i+1],:])
            grad_input = grad_input.reshape(grad_input.shape[0],
                                          grad_input.shape[1]*grad_input.shape[2], grad_input.shape[-1])
            fold = nn.Fold((shape[-2], shape[-1]), kernel_size =kernel_size, stride = stride, padding = padding )
            grad_input = fold(grad_input)

            return grad_input, None, None, None, None, None

class high_order_input(nn.Module):
    def __init__(self, kernel_size, stride, padding, TPCMs_r):
        super().__init__()
        self.func = HT_creation
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.TPCMs_r = TPCMs_r


    def forward(self, input):
        return self.func.apply(input,
                          self.kernel_size,
                          self.stride,
                          self.padding,
                          self.TPCMs_r
                          )


class HO_conv(nn.Module):
    def __init__(self, in_chan , out_chan, kernel_size, stride, padding, groups, TPCMs_r):
        super().__init__()
        self.kernel_size = kernel_size
        self.padding =  padding
        self.stride = stride
        self.TPCMs_r = TPCMs_r
        self.H_kernel_size = 0
        for i in range(len(self.TPCMs_r)):

            size = self.TPCMs_r[i][0].shape[0]

            self.H_kernel_size += size
        self.conv =  nn.Sequential( high_order_input( self.kernel_size,
                                                self.stride,
                                                self.padding,
                                                self.TPCMs_r),
                                    nn.Conv2d(in_chan, out_chan,
                                              kernel_size=(self.H_kernel_size,1),
                                              stride = 1,
                                              groups = groups, bias = False))
    def forward(self,input):
        in_shape = input.shape
        x = self.conv(input)
        x =  x.reshape(x.shape[0],
                        x.shape[1],
                      (in_shape[-2]-self.kernel_size[0]+ 2* self.padding[0])//self.stride[0] + 1,
                      (in_shape[-1]-self.kernel_size[1]+ 2* self.padding[1])//self.stride[1] + 1 )
        return x

In [ ]:




class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img


train_transform = transforms.Compose([

    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    # transforms.AutoAugment(AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761]),
    # Cutout(1, 16),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

# Load CIFAR-100 dataset with data augmentation
train_dataset = CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_dataset = CIFAR100(root='./data', train=False, download=True, transform=test_transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)


num_classes = 100
k = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class squeeze(nn.Module):
    def __init__(self, in_chan, reduce):
        super(squeeze, self).__init__()


        self.squ = nn.Sequential(
                                    nn.AdaptiveAvgPool2d(1),
                                    nn.Conv2d(in_chan, in_chan//reduce,1),
                                    nn.ReLU(),
                                    nn.Conv2d(in_chan//reduce,in_chan, 1),
                                    nn.Sigmoid()
                                    )
    def forward(self, input):
        x = self.squ(input)*input
        return x

class Volconv(nn.Module):
    def __init__(self, in_chan,out_chan, kernel_size, stride, padding, groups):
        super(Volconv, self).__init__()
        input_size = int(kernel_size[0]*kernel_size[1])
        # PM1_Full = torch.arange(input_size)[:,None]
        TRM2 = [[2],[1,1]]
        PM2_Full = PM_creation(2, input_size, TRM2)
        # TRM3 = [[3],[1,2],[2, 1], [ 1, 1, 1]]  # Third order Toe
        # PM3_Full = PM_creation(3, input_size, TPE3)
        TPCMs_r = [[PM2_Full]]
        # PCMs_r = PCM_creation(PM2_Full, PM3_Full)

        self.Second_conv = nn.Sequential(

                                   HO_conv(in_chan,  out_chan,
                                              kernel_size  ,
                                              stride  , padding, groups, TPCMs_r
                                              ), )


        self.conv = nn.Sequential(
                                   nn.Conv2d(in_chan, out_chan, kernel_size,
                                              stride , padding, groups = groups, bias = False))

    def forward(self, input):
        x = self.Second_conv(input)+self.conv(input)

        return x
class HLA(nn.Module):
    def __init__(self, in_chan, kernel_size, pooling_size, padding,  in_size, reduce):
        super(HLA, self).__init__()
        up_size1 = in_size[0]//pooling_size[0]
        up_size2 = in_size[1]//pooling_size[1]

        self.Vol1= nn.Sequential(   nn.BatchNorm2d( in_chan),
                                    nn.ReLU(),

                                    nn.AdaptiveAvgPool2d((pooling_size[0], pooling_size[1])),

                                    nn.Conv2d(in_chan, in_chan//reduce, kernel_size=1,
                                                stride=1 , padding = 0, groups = 1),
                                    nn.BatchNorm2d( in_chan//reduce),

                                    Volconv(in_chan//reduce,in_chan//reduce, kernel_size, (1, 1), padding, 1),

                                    nn.ReLU(),
                                    nn.BatchNorm2d( in_chan//reduce),

                                    nn.Conv2d(in_chan//reduce, in_chan, kernel_size=1,
                                                stride=1 , padding = 0, groups = 1),

                                    nn.Sigmoid(),

                                    )
        self.sq = nn.Sequential(
                                    nn.AdaptiveAvgPool2d(1),
                                    nn.Conv2d(in_chan, in_chan//reduce, 1),
                                    nn.ReLU(),
                                    nn.Conv2d(in_chan//reduce, in_chan, 1),
                                    nn.Sigmoid()
                                    )


        self.up = nn.Upsample(scale_factor= (up_size1,up_size2) )



    def forward(self, input):
        x1 = self.Vol1(input)
        x2 = self.sq(input)
        x3 = torch.mean(x2, dim=1, keepdim= True)
        x2 = x3 - nn.ReLU()(x3 - x2)
        x = x2 + x1 - x2*x1
        x = self.up(x)*input
        return x

class Branch(nn.Module):
    def __init__(self,
                  in_cha,
                  out_cha,
                  kernel_size,
                  stride,
                  padding,
                  in_size
                  ):

        super(Branch, self).__init__()

        self.shortcut = nn.Sequential()
        if in_cha != out_cha or stride != 1:
            self.shortcut = nn.Sequential( nn.Conv2d(in_cha,
                        out_cha,
                        kernel_size=1,
                        stride= stride  , padding=0,
                      bias = False),)

        self.Branch1_1 = nn.Sequential(
                                  nn.BatchNorm2d( in_cha),
                                  nn.ReLU(),

                                  nn.Conv2d(in_cha,
                                            out_cha,
                                            kernel_size = 3,
                                            stride= 1, padding = 1,
                                            bias = False),

                                  )


        self.Branch1_2 = nn.Sequential(
                                  nn.BatchNorm2d(out_cha),
                                  nn.ReLU(),

                                  nn.Conv2d( out_cha,
                                           out_cha,
                                            kernel_size = 3,
                                            stride= stride, padding = 1 ,
                                            bias = False),

                             )




    def forward(self, input):
        x = self.Branch1_1(input)
        x =  self.Branch1_2(x) + self.shortcut(input)
        return x
class Wide_net(nn.Module):
    def __init__(self, num_classes):
        super(Wide_net, self).__init__()

        self.start_covlayer = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1, 1, bias= False),

           )

        self.layer1 = nn.Sequential(

                                    Branch(16, 16*k,  3, 1, 1,32),
                                    HLA(16*k,  (3, 3), (2, 2), (1, 1), (32, 32), 16),

                                    Branch(16*k, 16*k,  3, 1, 1, 32),
                                    HLA(16*k,  (3, 3),(2, 2), (1, 1), (32, 32), 16),

                                    # Branch(16*k, 16*k,  3, 1, 1, 32),
                                    # HLA(16*k,  (3, 3), (2, 2), (1, 1), (32, 32), 16),

                                    # Branch(16*k, 16*k,  3, 1, 1, 32),
                                    # HLA(16*k,  (3, 3), (2, 2), (1, 1), (32, 32), 16),





                                    )


        self.layer2 = nn.Sequential(

                                    Branch(16*k, 32*k,  3, 2, 1,16),
                                    HLA(32*k,  (3, 3), (2, 2), (1, 1), (16, 16), 16),

                                    Branch(32*k, 32*k,3,1, 1, 16),
                                    HLA(32*k,  (3, 3), (2, 2), (1, 1), (16, 16), 16),


                                    # Branch(32*k, 32*k,3,1, 1, 16),
                                    #HLA(32*k,  (3, 3), (2, 2), (1, 1), (16, 16), 16),

                                    # Branch(32*k, 32*k,3,1, 1, 16),
                                    #HLA(32*k,  (3, 3), (2, 2), (1, 1), (16, 16), 16),



                                    # Branch(32*k, 32*k,3,1, 1, 16),
                                    #HLA(32*k,  (3, 3), (2, 2), (1, 1), (16, 16), 16),


                                    # Branch(32*k, 32*k,3,1, 1, 16),
                                    # HLA(32*k,  (3, 3), (2, 2), (1, 1), (16, 16), 16),

                                    )






        self.layer3 = nn.Sequential(

                                        Branch(32*k, 64*k,  3, 2, 1, 8),
                                        squeeze(64*k,16),

                                        Branch(64*k, 64*k, 3,1, 1, 8),

                                        squeeze(64*k,16),

                                        nn.BatchNorm2d(64*k),

                                        nn.ReLU(),

                                        nn.AdaptiveAvgPool2d(1) ,


                                            )


        self.fc = nn.Sequential(

        nn.Linear(64*k, num_classes),

        )


    def forward(self, x):
        out = self.start_covlayer(x)

        out = self.layer1(out)

        out = self.layer2(out)

        out = self.layer3(out)

        out = torch.flatten(out, start_dim=1)
        out = self.fc(out)
        return out


model = Wide_net(num_classes).to(device)


total_step = len(train_loader)
num_epochs = 200


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), momentum = 0.9, lr= 0.1,  weight_decay = 0.0005, nesterov=True)

scheduler1 = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120,160], gamma= 0.2)

total_params = sum(param.numel() for param in model.parameters())

with open('log_WRN_16_8_1.txt', 'w') as f:
    f.write('Epochs\tTraining Loss\tTraining_acc\tTesting_acc\n')

for epoch in range(num_epochs):
    model.train()
    model.training = True
    train_total = 0
    train_correct = 0
    for i, (images, labels) in enumerate(train_loader):

        # Move tensors to the configured device

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)


        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total +=labels.size(0)
        train_correct +=(predicted == labels).sum().item()

    scheduler1.step()
    train_acc = 100 * train_correct/train_total

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, train_acc: {:.3f} %'
                    .format(epoch+1, num_epochs, i+1, total_step, loss.item(), train_acc))


    # Validation
    correct = 0
    total = 0
    model.eval()
    model.training = False

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))
        with open('log_WRN_16_8_1.txt', 'a') as f:
            f.write('[{}/{}]\t{:.4f}\t        {:.3f} %\t      {:.2f} %\n'.format(
                epoch+1,num_epochs, loss.item(), train_acc, 100 * correct / total))



100%|██████████| 169001437/169001437 [00:02<00:00, 63107423.23it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/200], Step [391/391], Loss: 3.2563, train_acc: 11.430 %
Accuracy of the network on the 10000 test images: 15.7 %
Epoch [2/200], Step [391/391], Loss: 2.6777, train_acc: 25.082 %
Accuracy of the network on the 10000 test images: 27.79 %
Epoch [3/200], Step [391/391], Loss: 2.1976, train_acc: 36.474 %
Accuracy of the network on the 10000 test images: 36.49 %
Epoch [4/200], Step [391/391], Loss: 1.8147, train_acc: 44.496 %
Accuracy of the network on the 10000 test images: 41.51 %
Epoch [5/200], Step [391/391], Loss: 1.6855, train_acc: 50.368 %
Accuracy of the network on the 10000 test images: 40.38 %
Epoch [6/200], Step [391/391], Loss: 1.5218, train_acc: 54.448 %
Accuracy of the network on the 10000 test images: 46.47 %
Epoch [7/200], Step [391/391], Loss: 1.3196, train_acc: 57.970 %
Accuracy of the network on the 10000 test images: 53.07 %
Epoch [8/200], Step [391/391], Loss: 1.6762, train